In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.nn.functional as F

# Define a simple classifier on top of BERT
class RecipeClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(RecipeClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Define your recipe texts
recipes = [
    "1 tablespoons extra virgin olive oil. 1 cup chopped yellow onion. 3 cloves garlic (minced).",
    "1 cup chopped carrots. 2 cups sliced mushrooms. 1/2 cup diced tomatoes.",
    "1 cup cooked chickpeas. 1/2 cup chopped spinach. 1 tablespoon curry powder."
]

# Tokenize and encode the recipe texts
max_length = 128  # Maximum sequence length for BERT
encoded_inputs = tokenizer(recipes, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

# Instantiate the recipe classifier model
num_classes = 2  # Vegetarian or non-vegetarian
recipe_classifier = RecipeClassifier(bert_model, num_classes)

# Perform classification
with torch.no_grad():
    logits = recipe_classifier(encoded_inputs['input_ids'], encoded_inputs['attention_mask'])
    probabilities = F.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1)

# Print the predicted labels for the recipes
for i, recipe in enumerate(recipes):
    print(f"Recipe: {recipe}")
    print(f"Predicted Label: {'Vegetarian' if predicted_labels[i].item() == 0 else 'Non-Vegetarian'}")


---

# Embeddings on recipes with BERT, Neural Network for classification